In [1]:
#import everything that you can
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os
import sys
import openai
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
import glob

In [2]:
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"]  = os.getenv("OPENAI_API_KEY") 

In [75]:
#basic chatbot
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [76]:
# Initialize the updated ChatOpenAI model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

# Define the messages
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data")
]

# Get the response from the model using the new 'invoke' method
response = chat.invoke(messages)

# Print the content of the response
print(response.content, end='\n')

Sure! Here is an example Python script that trains a simple neural network on simulated data using the TensorFlow library:

```python
import numpy as np
import tensorflow as tf

# Generate simulated data
np.random.seed(0)
X = np.random.rand(1000, 2)
y = np.array([1 if x[0] + x[1] > 1 else 0 for x in X])

# Define the neural network architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print(f'Loss: {loss}, Accuracy: {accuracy}')
```

In this script:
1. We generate simulated data where the target variable is 1 if the sum of the two input features is greater than 1, otherwise 0.
2. We define a simple neural network with one hidden layer of 32 neurons a

In [6]:
#lets feed it our data

In [7]:
filepath = os.getcwd() + "\datasets"
csv_files = glob.glob(os.path.join(filepath, "*.csv"))
file1 = csv_files[0]
file1

'C:\\Users\\gauur\\projects\\WEB SCRAPE\\Indeed-Job-Scraping\\datasets\\combined_AI_Cincinnati_data.csv'

In [8]:
from langchain.document_loaders.csv_loader import CSVLoader

In [9]:
loader = CSVLoader(file1)
loader

In [10]:
data = loader.load()
data

[Document(metadata={'source': 'C:\\Users\\gauur\\projects\\WEB SCRAPE\\Indeed-Job-Scraping\\datasets\\combined_AI_Cincinnati_data.csv', 'row': 0}, page_content='Unnamed: 0: 0\nJob Title: Data Scientist, Masters Degree (Machine Learning and Operations Research)\nCompany: Procter & Gamble\nLocation: Cincinnati, OH 45202Â\xa0(Mount Adams area)\nSalary($): Not Provided\nJob-Type: Not Provided\nJob-Description: As a Data Scientist for P&G you will drive productivity and impact our business results across a variety of platforms.\nRaw_Link: https://www.indeed.com/rc/clk?jk=664d97a5eb67e73b&bb=n5eZx6LO6ip-TBsjA4uBbzG2WbHrMI5tCGEOtjWR4pWWTKzQQG3okFkGF4S18MLmPPOaiU2GJJC8WreZvqffVrHTo1mbyQXxSGXjuXwT48N-8FKggiOGcA%3D%3D&xkcb=SoDZ67M3-F9spXyD1Z0LbzkdCdPP&fccid=2da0dedf6df97194&vjs=3\nScraped_On: 2024-07-13'),
 Document(metadata={'source': 'C:\\Users\\gauur\\projects\\WEB SCRAPE\\Indeed-Job-Scraping\\datasets\\combined_AI_Cincinnati_data.csv', 'row': 1}, page_content='Unnamed: 0: 1\nJob Title: Exper

In [20]:
len(data)

75

In [50]:
max_rows = 38  # Adjust this number based on your needs and the data size
truncated_data = data[:max_rows]
truncated_data

[Document(metadata={'source': 'C:\\Users\\gauur\\projects\\WEB SCRAPE\\Indeed-Job-Scraping\\datasets\\combined_AI_Cincinnati_data.csv', 'row': 0}, page_content='Unnamed: 0: 0\nJob Title: Data Scientist, Masters Degree (Machine Learning and Operations Research)\nCompany: Procter & Gamble\nLocation: Cincinnati, OH 45202Â\xa0(Mount Adams area)\nSalary($): Not Provided\nJob-Type: Not Provided\nJob-Description: As a Data Scientist for P&G you will drive productivity and impact our business results across a variety of platforms.\nRaw_Link: https://www.indeed.com/rc/clk?jk=664d97a5eb67e73b&bb=n5eZx6LO6ip-TBsjA4uBbzG2WbHrMI5tCGEOtjWR4pWWTKzQQG3okFkGF4S18MLmPPOaiU2GJJC8WreZvqffVrHTo1mbyQXxSGXjuXwT48N-8FKggiOGcA%3D%3D&xkcb=SoDZ67M3-F9spXyD1Z0LbzkdCdPP&fccid=2da0dedf6df97194&vjs=3\nScraped_On: 2024-07-13'),
 Document(metadata={'source': 'C:\\Users\\gauur\\projects\\WEB SCRAPE\\Indeed-Job-Scraping\\datasets\\combined_AI_Cincinnati_data.csv', 'row': 1}, page_content='Unnamed: 0: 1\nJob Title: Exper

In [51]:
len(truncated_data)

38

In [52]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import ChatMessage

In [53]:
formatted_data = "\n".join([str(row) for row in truncated_data])
formatted_data

"page_content='Unnamed: 0: 0\nJob Title: Data Scientist, Masters Degree (Machine Learning and Operations Research)\nCompany: Procter & Gamble\nLocation: Cincinnati, OH 45202Â\xa0(Mount Adams area)\nSalary($): Not Provided\nJob-Type: Not Provided\nJob-Description: As a Data Scientist for P&G you will drive productivity and impact our business results across a variety of platforms.\nRaw_Link: https://www.indeed.com/rc/clk?jk=664d97a5eb67e73b&bb=n5eZx6LO6ip-TBsjA4uBbzG2WbHrMI5tCGEOtjWR4pWWTKzQQG3okFkGF4S18MLmPPOaiU2GJJC8WreZvqffVrHTo1mbyQXxSGXjuXwT48N-8FKggiOGcA%3D%3D&xkcb=SoDZ67M3-F9spXyD1Z0LbzkdCdPP&fccid=2da0dedf6df97194&vjs=3\nScraped_On: 2024-07-13' metadata={'source': 'C:\\\\Users\\\\gauur\\\\projects\\\\WEB SCRAPE\\\\Indeed-Job-Scraping\\\\datasets\\\\combined_AI_Cincinnati_data.csv', 'row': 0}\npage_content='Unnamed: 0: 1\nJob Title: Experienced Data Scientist (Machine Learning and Operations Research)\nCompany: Procter & Gamble\nLocation: Cincinnati, OH 45202Â\xa0(Mount Adams are

In [54]:
custom_prompt = f"""
Here is the data from the CSV file:\n{formatted_data}\n\nPlease analyze this data and provide insights based on the following criteria:
1) Find the most  suitable job from this to a data science junior student who is also minoring in statistics and a little focus on biological science.
2) No matter the location, the student can move from one location to another.
3) Also, please share the link to apply to each of these jobs as well
4) Check the web to see if they hire international students. Please include this in the response as well for each company.
"""

In [55]:
llm = ChatOpenAI(model="gpt-3.5-turbo")
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000022622371810>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000226217A2C50>, openai_api_key='sk-proj-6LAJYeahEuEBghXVIzLfT3BlbkFJee7iZtcx8QyDqXyS695x', openai_proxy='')

In [63]:
message = ChatMessage(role="user", content=custom_prompt)
response = llm([message])
answer1 = response.content
print(response.content,end='\n')

Based on the criteria provided, the most suitable job for a data science junior student minoring in statistics with a focus on biological science would be the "Computer Vision Engineer" position at GE Aviation in Evendale, OH. This role involves software development projects related to cloud computing, parallelization, computational optimization, and object-oriented design, which align well with a background in data science and a focus on biological science.

Here is the link to apply for the job: [Computer Vision Engineer at GE Aviation](https://www.indeed.com/rc/clk?jk=5a717b036e81c439&bb=vB7T1JJE9rnggg_eofGIRfBNHJ3Klg7OptJzpPwrBZR9jtnweZFHNjOaEUeZpmSue0oPJnJFSKWQUafU4FqdXHgDxoMOa7yGyj6P25zHmL7pstwCekcOEQ%3D%3D&xkcb=SoCg67M3-F9ocpQiNJ0JbzkdCdPP&fccid=aac6f2070b0f3775&vjs=3)

In terms of hiring international students, GE Aviation does hire international candidates for certain positions. It is recommended to check the company's official website or contact their HR department for specif

In [42]:
#on the remaining data

In [57]:
max_rows = len(data)  # Adjust this number based on your needs and the data size
other = data[38:max_rows]
other

[Document(metadata={'source': 'C:\\Users\\gauur\\projects\\WEB SCRAPE\\Indeed-Job-Scraping\\datasets\\combined_AI_Cincinnati_data.csv', 'row': 38}, page_content='Unnamed: 0: 8\nJob Title: SAP BTP Consultant\nCompany: Accenture\nLocation: Cincinnati, OH 45202Â\xa0(Central Business District area)\nSalary($): Not Provided\nJob-Type: Full-time\nJob-Description: We help our clients apply intelligence to set their business apart and make them more proactive, predictive and productive Ã¢â‚¬â€œ the power of the intelligentÃ¢â‚¬Â¦\nRaw_Link: https://www.indeed.com/rc/clk?jk=c1f7583df0a1a9df&bb=vB7T1JJE9rnggg_eofGIRQX-kBaeL6wOzYSCLTE_7iE_VZ5tRvaysUOxHKYraJiigjWSFQRj6Vam-vKbNVfIxG4Kf4KzImJBN3dnrJ5_3x5BHpuv61zYDA%3D%3D&xkcb=SoDu67M3-F9ocpQiNJ0DbzkdCdPP&fccid=a4e4e2eaf26690c9&vjs=3\nScraped_On: 2024-07-13'),
 Document(metadata={'source': 'C:\\Users\\gauur\\projects\\WEB SCRAPE\\Indeed-Job-Scraping\\datasets\\combined_AI_Cincinnati_data.csv', 'row': 39}, page_content='Unnamed: 0: 9\nJob Title: Senio

In [58]:
len(other)

37

In [59]:
formatted_data = "\n".join([str(row) for row in other])
formatted_data

"page_content='Unnamed: 0: 8\nJob Title: SAP BTP Consultant\nCompany: Accenture\nLocation: Cincinnati, OH 45202Â\xa0(Central Business District area)\nSalary($): Not Provided\nJob-Type: Full-time\nJob-Description: We help our clients apply intelligence to set their business apart and make them more proactive, predictive and productive Ã¢â‚¬â€œ the power of the intelligentÃ¢â‚¬Â¦\nRaw_Link: https://www.indeed.com/rc/clk?jk=c1f7583df0a1a9df&bb=vB7T1JJE9rnggg_eofGIRQX-kBaeL6wOzYSCLTE_7iE_VZ5tRvaysUOxHKYraJiigjWSFQRj6Vam-vKbNVfIxG4Kf4KzImJBN3dnrJ5_3x5BHpuv61zYDA%3D%3D&xkcb=SoDu67M3-F9ocpQiNJ0DbzkdCdPP&fccid=a4e4e2eaf26690c9&vjs=3\nScraped_On: 2024-07-13' metadata={'source': 'C:\\\\Users\\\\gauur\\\\projects\\\\WEB SCRAPE\\\\Indeed-Job-Scraping\\\\datasets\\\\combined_AI_Cincinnati_data.csv', 'row': 38}\npage_content='Unnamed: 0: 9\nJob Title: Senior Cloud Security Engineer\nCompany: Procter & Gamble\nLocation: Cincinnati, OH 45202Â\xa0(Mount Adams area)\nSalary($): $105,000 - $150,000 a yea

In [60]:
llm = ChatOpenAI(model="gpt-3.5-turbo")
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000022621C264D0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000022622384A10>, openai_api_key='sk-proj-6LAJYeahEuEBghXVIzLfT3BlbkFJee7iZtcx8QyDqXyS695x', openai_proxy='')

In [64]:
message = ChatMessage(role="user", content=custom_prompt)
response2 = llm([message])
answer2 = response2.content
print(response2.content,end='\n')

Based on the criteria provided, the most suitable job for a data science junior student who is minoring in statistics and has a focus on biological science would be the "Computer Vision Engineer" position at GE Aviation in Evendale, OH. This role involves software development projects that include cloud computing, parallelization, computational optimization, and object-oriented design, which align well with a background in data science and statistics. Additionally, the focus on computer vision ties in with the student's interest in biological science.

Here is the link to apply for the job: [Computer Vision Engineer at GE Aviation](https://www.indeed.com/rc/clk?jk=5a717b036e81c439&bb=vB7T1JJE9rnggg_eofGIRfBNHJ3Klg7OptJzpPwrBZR9jtnweZFHNjOaEUeZpmSue0oPJnJFSKWQUafU4FqdXHgDxoMOa7yGyj6P25zHmL7pstwCekcOEQ%3D%3D&xkcb=SoCg67M3-F9ocpQiNJ0JbzkdCdPP&fccid=aac6f2070b0f3775&vjs=3)

In terms of international student hiring policies, it is recommended to visit the company's official website or conta

In [65]:
#merging both answers:

In [73]:
final_answer = "This is the first job:\n" + answer1 + "\n\n" + "----" + "\n\n" + "And, this is the second job: \n" + answer2
final_answer

'This is the first job:\nBased on the criteria provided, the most suitable job for a data science junior student minoring in statistics with a focus on biological science would be the "Computer Vision Engineer" position at GE Aviation in Evendale, OH. This role involves software development projects related to cloud computing, parallelization, computational optimization, and object-oriented design, which align well with a background in data science and a focus on biological science.\n\nHere is the link to apply for the job: [Computer Vision Engineer at GE Aviation](https://www.indeed.com/rc/clk?jk=5a717b036e81c439&bb=vB7T1JJE9rnggg_eofGIRfBNHJ3Klg7OptJzpPwrBZR9jtnweZFHNjOaEUeZpmSue0oPJnJFSKWQUafU4FqdXHgDxoMOa7yGyj6P25zHmL7pstwCekcOEQ%3D%3D&xkcb=SoCg67M3-F9ocpQiNJ0JbzkdCdPP&fccid=aac6f2070b0f3775&vjs=3)\n\nIn terms of hiring international students, GE Aviation does hire international candidates for certain positions. It is recommended to check the company\'s official website or contact 

In [74]:
print(final_answer,end='\n')

This is the first job:
Based on the criteria provided, the most suitable job for a data science junior student minoring in statistics with a focus on biological science would be the "Computer Vision Engineer" position at GE Aviation in Evendale, OH. This role involves software development projects related to cloud computing, parallelization, computational optimization, and object-oriented design, which align well with a background in data science and a focus on biological science.

Here is the link to apply for the job: [Computer Vision Engineer at GE Aviation](https://www.indeed.com/rc/clk?jk=5a717b036e81c439&bb=vB7T1JJE9rnggg_eofGIRfBNHJ3Klg7OptJzpPwrBZR9jtnweZFHNjOaEUeZpmSue0oPJnJFSKWQUafU4FqdXHgDxoMOa7yGyj6P25zHmL7pstwCekcOEQ%3D%3D&xkcb=SoCg67M3-F9ocpQiNJ0JbzkdCdPP&fccid=aac6f2070b0f3775&vjs=3)

In terms of hiring international students, GE Aviation does hire international candidates for certain positions. It is recommended to check the company's official website or contact their H